In [12]:
import requests 
import urllib
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [15]:
root = 


soup = BeautifulSoup(page.content, 'html.parser')

In [28]:
page.raise_for_status()

In [67]:
entries = page.json()['items']


In [53]:
[0]['stationReference']

'1029TH'

In [121]:
import numpy as np
import datetime as dt
import pandas as pd
from collections import defaultdict  

In [201]:
class MeasurementTool():
    def __init__(self,):
        self.minutes_arr = ['00','15','30','45']
        self.root='https://environment.data.gov.uk/flood-monitoring'
        self.page_url = self.root + '/id/stations/'
        self.page = requests.get(self.page_url)
        self.entries = self.page.json()['items']
        self.stations = self.create_stations_list()
        self.structured_data = defaultdict(list)
    
    def retrieve_measurement(self,station_id):
        station_url = self.page_url + station_id
        latest_possible_reading_time = self.get_rounded_timestamp()
        last_24_hrs = self.subtract_24_hours(latest_possible_reading_time)
        station_page = requests.get(station_url + "/readings?since=" + last_24_hrs)
        station_data = station_page.json()["items"]

        for item in station_data:
            measure_name = "-".join(item["measure"].split("/")[-1].split("-")[1:])  # Extract measure name
            timestamp = item["dateTime"]
            value = item["value"]
            self.structured_data[measure_name].append({"timestamp": timestamp, "value": value})

        return self.structured_data

    def get_rounded_timestamp(self):
        """Returns the current timestamp rounded down to the nearest 15-minute increment."""
        now = dt.datetime.now(dt.timezone.utc)
        rounded_minutes = now.minute - (now.minute % 15)  # Round down to nearest 15-minute mark
        rounded_time = now.replace(minute=rounded_minutes, second=0, microsecond=0)
        return rounded_time.strftime("%Y-%m-%dT%H:%M:%SZ")

    def subtract_24_hours(self,timestamp: str) -> str:
        """Subtracts 24 hours from a given timestamp and returns it in the same format."""
        datetime = dt.datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
        new_dt = datetime - dt.timedelta(hours=24)
        return new_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
         
    def create_stations_list(self):
        stations = []
        station_ids = []
        for entry in self.entries: 
            stations.append(entry['@id'])
            station_ids.append(entry['stationReference'])
            break
        return station_ids
    
    def create_widgets(self):
        station_selector = widgets.Dropdown(
            options=self.create_stations_list(),  # List of station IDs
            description="Station:",
            style={'description_width': 'initial'}
        )

        # Define the button widget
        run_button = widgets.Button(description="Retrieve Data")

        # Define the output area
        output = widgets.Output()

        # Define what happens when the button is clicked
        def on_button_click(b):
            with output:
                output.clear_output()  # Clear previous output
                station_id = station_selector.value
                print(f"Fetching data for {station_id}...")
                # Call your function here (e.g., retrieve_measurement(station_id))
                data = self.retrieve_measurement(station_id)
                print(data)

        # Link the button to the function
        run_button.on_click(on_button_click)

        # Display widgets
        display(station_selector, run_button, output)        

In [197]:
tool = MeasurementTool()


defaultdict(list,
            {'level-downstage-i-15_min-mASD': [{'timestamp': '2025-03-07T12:00:00Z',
               'value': -0.148},
              {'timestamp': '2025-03-07T12:15:00Z', 'value': -0.149},
              {'timestamp': '2025-03-07T12:30:00Z', 'value': -0.149},
              {'timestamp': '2025-03-07T12:45:00Z', 'value': -0.151},
              {'timestamp': '2025-03-07T13:00:00Z', 'value': -0.148},
              {'timestamp': '2025-03-07T13:15:00Z', 'value': -0.149},
              {'timestamp': '2025-03-07T13:30:00Z', 'value': -0.149},
              {'timestamp': '2025-03-07T13:45:00Z', 'value': -0.15},
              {'timestamp': '2025-03-07T14:00:00Z', 'value': -0.15},
              {'timestamp': '2025-03-07T14:15:00Z', 'value': -0.151},
              {'timestamp': '2025-03-07T14:30:00Z', 'value': -0.151},
              {'timestamp': '2025-03-07T14:45:00Z', 'value': -0.151},
              {'timestamp': '2025-03-07T15:00:00Z', 'value': -0.151},
              {'timestamp'

In [198]:
data = tool.retrieve_measurement('1029TH')

In [200]:
data.keys()

dict_keys(['level-downstage-i-15_min-mASD', 'level-stage-i-15_min-mASD'])

In [196]:
tool = MeasurementTool()
tool.create_widgets()

Dropdown(description='Station:', options=('1029TH', 'E2043', '52119', 'E21136', '2067', '48143', '720215', 'F1…

Button(description='Retrieve Data', style=ButtonStyle())

Output()

In [173]:
pd.DataFrame(measurements)['level-downstage-i-15_min-mASD'][0]['timestamp']


'2025-03-07T02:00:00Z'

In [181]:
import ipywidgets as widgets
from IPython.display import display

# Define the dropdown widget
tool.c

Dropdown(description='Station:', options=('1491TH', '1029TH', '2384AB', '3072XY'), style=DescriptionStyle(desc…

Button(description='Retrieve Data', style=ButtonStyle())

Output()